<a href="https://colab.research.google.com/github/CharlotteY2003/MIT-Futuremakers/blob/main/Gender_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Warning: Not fully completed, as accuracy is not as high as Kaggle notebook due to the fact that this notebook was not a complete code by code copy of it (tried to do notebook without looking first, then added some of Kaggle notebook's lines). Will refine over the weekend. Also computing error - took too long to compile so unsure of results. 

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import *
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split, KFold
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import seaborn as sns
from sklearn.metrics import confusion_matrix
from numpy.random import seed

In [2]:
data = pd.read_csv('age_gender.csv')
data.head()

,age,ethnicity,gender,img_name,pixels
0,1,2,0,20161219203650636.jpg.chip.jpg,129 128 128 126 127 130 133 135 139 142 145 14...
1,1,2,0,20161219222752047.jpg.chip.jpg,164 74 111 168 169 171 175 182 184 188 193 199...
2,1,2,0,20161219222832191.jpg.chip.jpg,67 70 71 70 69 67 70 79 90 103 116 132 145 155...
3,1,2,0,20161220144911423.jpg.chip.jpg,193 197 198 200 199 200 202 203 204 205 208 21...
4,1,2,0,20161220144914327.jpg.chip.jpg,202 205 209 210 209 209 210 211 212 214 218 21...


In [3]:
#Convert each item of pixels into array
def string_to_arr(X, width, height):
  X = X.apply(lambda x: np.array(x.split(), dtype='float32'))
  X = np.array(X)/255.0
  X = np.array([X[i].reshape(width, height, 1) for i in range(X.shape[0])])
  return X

In [4]:
def data_preprocess(data):
  data = data[data['age'] >= 18]
  data.reset_index(drop=True, inplace=True)
  data = data.dropna()

  num_pixels = len(data['pixels'][0].split())
  dimensions = int(np.sqrt(num_pixels))
  width = dimensions
  height = dimensions

  X = data.pixels
  y = np.array(data.gender)

  X = string_to_arr(X, width, height)

  return X,y

In [5]:
def data_aug(X_val, y_val, X_test, y_test):
  train_data_gen = ImageDataGenerator(rotation_range=30,
                                      width_shift_range = 10,
                                      brightness_range=[.8,1.2],
                                      zoom_range = [.8,1.2],
                                      rescale=1/255)
  val_data_gen = ImageDataGenerator(rescale=1/255)
  test_data_gen = ImageDataGenerator(rescale=1/255)

  #set_seed(11)
  #random.seed(11)
  np.random.seed(11)

  val_data = val_data_gen.flow(X_val, y_val, shuffle = False, seed= 11)
  test_data = test_data_gen.flow(X_test, y_test, shuffle=False, seed=11)
  
  return val_data, test_data, train_data_gen

In [6]:
def build_model():
  model = Sequential()
  model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform', input_shape = (48,48,1,)))
  model.add(MaxPooling2D())
  model.add(BatchNormalization())
  
  model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPooling2D())
  model.add(Dropout(.3))
  model.add(BatchNormalization())

  model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPooling2D())
  model.add(Dropout(.3))
  model.add(BatchNormalization())
  
  model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu', kernel_initializer='he_uniform', padding='same'))
  model.add(MaxPooling2D())
  model.add(Dropout(.5))
  model.add(BatchNormalization())

  model.add(Flatten())
  model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
  model.add(Dense(128, activation='softmax', kernel_initializer='he_uniform')) #Why is ouput units not 1 or 2?

  #optimizer = SGD(learning_rate=.01, momentum=.9)
  model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
  
  return model

In [7]:
def train_test(model, X, y, train_data_gen, nsplit=10):
  
  histories = list()
  scores = list()
  kf = KFold(n_splits = nsplit, shuffle=True, random_state=2)

  es = EarlyStopping(min_delta=.01, patience=5, mode='min', restore_best_weights=True)
  checkpoint = ModelCheckpoint('weights.h5', mode = 'min', save_best_only=True)
  
  for traini, testi in kf.split(X,y):
    #print('hi')
    #X_train, X_val = X[traini], X[testi]
    #y_train, y_val = y[traini], y[testi]
    train_data = train_data_gen.flow(X[traini], y[traini], shuffle=False, seed=11)
    temp_val_data = train_data_gen.flow(X[testi], y[testi], shuffle=False, seed=11)
    history = model.fit(train_data, epochs = 20, batch_size = 32, validation_data = temp_val_data, 
                        verbose = 1, steps_per_epoch = 10,
                        callbacks=[es,checkpoint])
  
    _, acc = model.evaluate(temp_val_data)
    scores.append(acc)
    histories.append(history)

  model.load_weights('weights.h5')
  
  return scores, histories, model

In [8]:
def evaluate_performance(scores, histories):
  for i in range(len(histories)):
    plt.plot(histories[i].history['loss'], label='train', color='blue')
    plt.plot(histories[i].history['val_loss'], label='test', color='orange')
    plt.title('Model ' + str(i+1) + " loss")
    plt.show()

    plt.plot(histories[i].history['accuracy'], label = 'train', color='blue')
    plt.plot(histories[i].history['val_accuracy'], label = 'test', color='orange')
    plt.title('Model ' + str(i+1) + " accuracy")
    plt.show()

  plt.plot(scores)
  plt.title('Overall scores')
  plt.show()

In [9]:
def run_test(data):
  X,y = data_preprocess(data)
  model = build_model()
  
  X_train, X_te, y_train, y_te = train_test_split(X, y, test_size = .3, random_state = 2)
  X_val, X_test, y_val, y_test = train_test_split(X_te, y_te, test_size=.15, random_state=2)
  
  val_data, test_data, train_data_gen = data_aug(X_val, y_val, X_test, y_test)
  
  scores, histories, best_model = train_test(model, X_train, y_train, train_data_gen)
  evaluate_performance(scores, histories)
  
  final_train = np.append(X_train, X_val, axis=0)
  final_val = np.append(y_train, y_val, axis=0) #Shouldn't it be x,y?
  final_training_data = train_data_gen.flow(final_train, final_val, seed=11) #What's the point of this if we're not going to use it?

  history = best_model.fit(final_training_data, epochs = 50, batch_size = 32, validation_data = val_data, 
                        verbose = 1, steps_per_epoch = 10) #confused on this line
  print(best_model.evaluate(test_data))

  y_pred = best_model.predict_classes(test_data)

  cm = confusion_matrix(y_test, y_pred)
  sns.heatmap(cm, cmap = 'Greens', cbar=False, annot=True, fmt='d')

In [10]:
run_test(data)

ValueError: ignored